## 작업 개요
- 목표: 공공데이터포털 오름현황 API에서 오름명/위도/경도만 추출해 표로 정리하고, 필요시 Folium으로 지도 마커 생성.
- 보안: 인증키는 코드에 직접 넣지 말고 환경변수 `ODCLOUD_SERVICE_KEY` 또는 `.env`에 설정하세요. 예) `.env` 내용: `ODCLOUD_SERVICE_KEY=...`
- API 엔드포인트: 최신(2024-05-02) UDDI 사용 — `uddi:78ffcb9e-6d65-4650-8568-921c94d0d9ec`
- 출력: 오름명/위도/경도 컬럼만 가진 DataFrame, CSV 저장, (선택) Folium 지도 HTML 저장.


In [ ]:
# 기본 라이브러리
import os
import json
from urllib.parse import unquote

import pandas as pd
import requests

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 120)


In [ ]:
# 설정: 엔드포인트와 서비스 키 (환경변수 사용 권장)
BASE = 'https://api.odcloud.kr/api'
UDDI = 'uddi:78ffcb9e-6d65-4650-8568-921c94d0d9ec'  # 2024-05-02
ENDPOINT = f'/3082952/v1/{UDDI}'

# 인증키는 인코딩(%)/디코딩 키 모두 가능. 과다 인코딩을 방지하기 위해 먼저 디코딩하여 원문으로 정규화합니다.
raw_key = os.getenv('ODCLOUD_SERVICE_KEY', '')
if not raw_key:
    raise RuntimeError('환경변수 ODCLOUD_SERVICE_KEY 가 설정되지 않았습니다. .env 또는 환경변수로 설정하세요.')
def _normalize_key(k: str) -> str:
    for _ in range(3):
        nk = unquote(k)
        if nk == k:
            break
        k = nk
    return k
service_key = _normalize_key(raw_key)

url = f'{BASE}{ENDPOINT}'
params = {
    'serviceKey': service_key,
    'page': 1,
    'perPage': 1000,
    'returnType': 'JSON'
}
url, params


In [ ]:
# 데이터 요청 및 DataFrame 생성
resp = requests.get(url, params=params, timeout=30)
resp.raise_for_status()
payload = resp.json()

# odcloud 응답은 보통 { 'data': [...] } 형태이나, 변형될 수 있어 유연하게 처리
if isinstance(payload, dict):
    if 'data' in payload and isinstance(payload['data'], list):
        rows = payload['data']
    elif 'records' in payload and isinstance(payload['records'], list):
        rows = payload['records']
    else:
        # dict이지만 리스트 키를 못 찾은 경우: 값들 중 리스트를 탐색
        rows = next((v for v in payload.values() if isinstance(v, list)), [])
else:
    rows = payload if isinstance(payload, list) else []

df = pd.DataFrame(rows)
print('총 행 수:', len(df))
print('칼럼 목록:', list(df.columns))
df.head(3)


In [ ]:
# 칼럼 자동 선택 유틸: 오름명/위도/경도 유사명 매핑
def select_oreum_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols = list(df.columns)
    lower = {c: str(c).lower() for c in cols}

    def find_col(candidates):
        for cand in candidates:
            # 한국어/영문 혼합 가능성 고려
            for c in cols:
                lc = lower[c]
                if cand in c or cand in lc:
                    return c
        return None

    name_col = find_col(['오름', '명칭', '이름', '명'])
    lat_col  = find_col(['위도', 'latitude', 'lat', '(y', ' y', ' y)'])
    lon_col  = find_col(['경도', 'longitude', 'lon', '(x', ' x', ' x)'])

    if not all([name_col, lat_col, lon_col]):
        raise KeyError(f'필수 칼럼을 찾지 못했습니다. name={name_col}, lat={lat_col}, lon={lon_col}')

    out = df[[name_col, lat_col, lon_col]].copy()
    out.columns = ['오름명', '위도', '경도']

    # 숫자 변환 및 결측 제거
    out['위도'] = pd.to_numeric(out['위도'], errors='coerce')
    out['경도'] = pd.to_numeric(out['경도'], errors='coerce')
    out = out.dropna(subset=['위도', '경도'])

    # 한국 좌표 범위 간단 필터 (제주 근사 범위)
    out = out[(out['위도'].between(32.5, 34.5)) & (out['경도'].between(125.0, 129.0))]
    return out.reset_index(drop=True)


In [ ]:
# 표로 정리 + 저장
oreum_df = select_oreum_columns(df)
display(oreum_df.head(10))
csv_path = '제주오름_좌표.csv'
oreum_df.to_csv(csv_path, index=False)
print('CSV 저장:', csv_path, '| 행 수:', len(oreum_df))


## (선택) Folium 지도 마커 생성
- 표에서 추출한 좌표로 마커를 생성해 HTML 파일로 저장합니다.


In [ ]:
# !pip install folium  # 필요시 설치
import folium
from IPython.display import display

center = [33.38, 126.53]  # 제주 중심 근사
m = folium.Map(location=center, zoom_start=10, tiles='OpenStreetMap')

for _, r in oreum_df.iterrows():
    folium.Marker([r['위도'], r['경도']], popup=str(r['오름명'])).add_to(m)

# 코랩/주피터에서 바로 표시
display(m)


## Gradio 앱 개요
- 기능: 오름명을 검색하고 Folium 지도를 노트북 내에서 렌더링합니다.
- 데이터 소스: (우선순위) 업로드한 CSV > 데이터 URL(CSV/JSON) > 로컬 CSV("제주오름_좌표.csv") > ODCloud API.
- 인증: 환경변수 `ODCLOUD_SERVICE_KEY` 또는 `.env` 파일(루트)에 키를 저장해 사용합니다. 키는 과다 인코딩 방지를 위해 디코딩 정규화합니다.
- UI: 검색어, 클러스터 토글, CSV 업로드/URL, 지도/결과/표 미리보기


In [ ]:
# Gradio/Folium 앱 공통 임포트 및 유틸
import os
from typing import Tuple, Optional
from urllib.parse import unquote

import pandas as pd
import requests
import folium
from folium.plugins import MarkerCluster
import gradio as gr

JEJU_CENTER = (33.38, 126.53)

def _load_dotenv_simple(path: str = '.env') -> None:
    """간단한 .env 로더 (KEY=VALUE, 따옴표 없음)."""
    if not os.path.exists(path):
        return
    try:
        with open(path, 'r', encoding='utf-8') as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith('#'):
                    continue
                if '=' in line:
                    k, v = line.split('=', 1)
                    k, v = k.strip(), v.strip()
                    if k and v and k not in os.environ:
                        os.environ[k] = v
    except Exception:
        # best-effort
        pass

def _normalize_key(key: str) -> str:
    """퍼센트-인코딩된 키를 최대 3회까지 디코딩해 정규화합니다."""
    if not key:
        return key
    for _ in range(3):
        nk = unquote(key)
        if nk == key:
            break
        key = nk
    return key


### 데이터 로딩 함수
- 우선 로컬 CSV(`제주오름_좌표.csv`)가 있으면 사용합니다.
- 없으면 ODCloud API를 호출해 DataFrame을 생성합니다.
- 다양한 칼럼명 변형을 허용해 오름명/위도/경도(및 선택 칼럼)를 정규화합니다.


In [ ]:
def _select_oreum_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols = list(df.columns)
    lower = {c: str(c).lower() for c in cols}

    def find_col(cands):
        for cand in cands:
            for c in cols:
                lc = lower[c]
                if cand in str(c) or cand in lc:
                    return c
        return None

    # Required fields
    name_col = find_col(['오름', '오름명', '명칭', '이름', '명']) or '오름명'
    lat_col = find_col(['위도', 'latitude', 'lat', '(y', ' y', ' y)']) or '위도'
    lon_col = find_col(['경도', 'longitude', 'lon', '(x', ' x', ' x)']) or '경도'

    # Dataset-specific fields
    district_col = find_col(['읍면동'])
    addr_col = find_col(['소재지', '주소', '도로명', 'address'])
    parking_col = find_col(['주차장', 'parking'])
    toilet_col = find_col(['화장실', 'toilet', 'restroom'])
    elev_col = find_col(['표고(m)', '표고', '고도', 'elevation', 'height'])
    refdate_col = find_col(['데이터기준일자', '기준일자', '기준일', 'date'])

    keep = [name_col, lat_col, lon_col]
    opt = [c for c in [district_col, addr_col, parking_col, toilet_col, elev_col, refdate_col] if c]
    out = df[keep + opt].copy()
    rename = {name_col: '오름명', lat_col: '위도', lon_col: '경도'}
    if district_col: rename[district_col] = '읍면동'
    if addr_col: rename[addr_col] = '소재지'
    if parking_col: rename[parking_col] = '주차장'
    if toilet_col: rename[toilet_col] = '화장실'
    if elev_col: rename[elev_col] = '표고(m)'
    if refdate_col: rename[refdate_col] = '데이터기준일자'
    out = out.rename(columns=rename)

    # Cast numeric lat/lon and elevation
    out['위도'] = pd.to_numeric(out['위도'], errors='coerce')
    out['경도'] = pd.to_numeric(out['경도'], errors='coerce')
    if '표고(m)' in out.columns:
        out['표고(m)'] = pd.to_numeric(out['표고(m)'], errors='ignore')
    out = out.dropna(subset=['위도', '경도']).reset_index(drop=True)

    # Jeju bbox filter
    out = out[(out['위도'].between(32.5, 34.5)) & (out['경도'].between(125.0, 129.0))]

    # Ensure optional dataset columns exist
    for c2 in ['읍면동', '소재지', '주차장', '화장실', '표고(m)', '데이터기준일자']:
        if c2 not in out.columns:
            out[c2] = ''

    ordered = ['오름명', '위도', '경도', '읍면동', '소재지', '주차장', '화장실', '표고(m)', '데이터기준일자']
    extras = [c for c in out.columns if c not in ordered]
    out = out[ordered + extras]
    return out.reset_index(drop=True)


### 지도 렌더링 및 검색 함수
- Folium 지도를 생성하고, 팝업에 데이터셋 주요 항목(읍면동, 소재지, 주차장, 화장실, 표고, 기준일자)을 표시합니다.
- 파일 업로드/URL에서 데이터 로딩을 지원합니다.


In [ ]:
def _build_map(df: pd.DataFrame, use_cluster: bool) -> folium.Map:
    if len(df) == 0:
        return folium.Map(location=JEJU_CENTER, zoom_start=10, tiles='OpenStreetMap')
    lat, lon = df['위도'].mean(), df['경도'].mean()
    m = folium.Map(location=[lat, lon], zoom_start=11, tiles='OpenStreetMap')
    layer = MarkerCluster() if use_cluster else m
    if use_cluster:
        layer.add_to(m)
    for _, r in df.iterrows():
        name = str(r.get('오름명', ''))
        html_parts = [f"<h4 style='margin:4px 0'>{name}</h4>"]
        details = []
        def add(label):
            val = r.get(label, '')
            try:
                is_na = pd.isna(val)
            except Exception:
                is_na = False
            if is_na or str(val).strip()=='' :
                return
            v = str(val)
            if label in ['화장실','주차장'] and v in ('Y','N','y','n'):
                v = '예' if v.upper()=='Y' else '아니오'
            details.append(f"<div><b>{label}</b>: {v}</div>")
        for lab in ['읍면동','소재지','주차장','화장실','표고(m)','데이터기준일자']:
            add(lab)
        if details:
            html_parts.append('<div>'+''.join(details)+'</div>')
        popup_html = ''.join(html_parts)
        folium.Marker([r['위도'], r['경도']], popup=folium.Popup(popup_html, max_width=300)).add_to(layer)
    return m


### Gradio UI 구성 및 실행
- 검색/업데이트 버튼과 입력 변화(엔터/업로드/URL입력)에 반응합니다.
- 노트북/Colab에서는 자동으로 셀 아래에 렌더링됩니다.


In [ ]:
with gr.Blocks(title='제주 오름 지도 검색') as demo:
    gr.Markdown("""
    # 제주 오름 지도 검색 (Gradio)
    - 검색어로 오름명을 필터링하고 Folium 지도를 표시합니다.
    - 클러스터 옵션으로 다수 마커를 보기 쉽게 묶을 수 있습니다.
    """)
    with gr.Row():
        query = gr.Textbox(label='검색어 (오름명)', placeholder='예: 군산, 오름 ...', scale=3)
        cluster = gr.Checkbox(value=True, label='마커 클러스터')
        run_btn = gr.Button('검색/업데이트', variant='primary')
    with gr.Row():
        map_html = gr.HTML(label='지도')
    with gr.Row():
        info_text = gr.Textbox(label='검색 결과', interactive=False)
    with gr.Row():
        table = gr.Dataframe(label='오름 목록 (일부 미리보기)', wrap=True)
    def _run(q, c):
        html, df, info = search_and_render(q, c)
        return html, info, df
    run_btn.click(_run, inputs=[query, cluster], outputs=[map_html, info_text, table])
    query.submit(_run, inputs=[query, cluster], outputs=[map_html, info_text, table])
    gr.on(triggers=[demo.load], fn=_run, inputs=[query, cluster], outputs=[map_html, info_text, table])
# 노트북에서 실행 시, 아래가 인라인으로 렌더링됩니다.
demo.launch(debug=True, inline=True, share=False)
